In [1]:
import pandas as pd

In [62]:
airports = pd.read_json("airports.json",orient='index')

In [63]:
airports.head()

,city,country,elevation,iata,icao,lat,lon,name,state,tz
00AK,Anchor Point,US,450,,00AK,59.949200,-151.695999,Lowell Field,Alaska,America/Anchorage
00AL,Harvest,US,820,,00AL,34.864799,-86.770302,Epps Airpark,Alabama,America/Chicago
00AZ,Cordes,US,3810,,00AZ,34.305599,-112.165001,Cordes Airport,Arizona,America/Phoenix
00CA,Barstow,US,3038,,00CA,35.350498,-116.888000,Goldstone /Gts/ Airport,California,America/Los_Angeles
00CO,Briggsdale,US,4830,,00CO,40.622200,-104.344002,Cass Field,Colorado,America/Denver


In [64]:
airports = airports[airports['country'].isin(['US','CA'])]
airports = airports.drop(columns=['country','elevation','iata','tz'])
airports

,city,icao,lat,lon,name,state
00AK,Anchor Point,00AK,59.949200,-151.695999,Lowell Field,Alaska
00AL,Harvest,00AL,34.864799,-86.770302,Epps Airpark,Alabama
00AZ,Cordes,00AZ,34.305599,-112.165001,Cordes Airport,Arizona
00CA,Barstow,00CA,35.350498,-116.888000,Goldstone /Gts/ Airport,California
00CO,Briggsdale,00CO,40.622200,-104.344002,Cass Field,Colorado
...,...,...,...,...,...,...
XS92,Paradise,XS92,33.146000,-97.712502,Jackson /Bill/ Airport,Texas
XS93,Santa Elena,XS93,26.716999,-98.558601,Diamond O Ranch Airport,Texas
XS94,San Ygnacio,XS94,27.111401,-99.422501,Corralitos Airport,Texas
XS95,Schulenburg,XS95,29.676901,-96.970497,Marty Ranch Airport,Texas


In [65]:
airports.to_json('USCAairports.json',orient='index')

In [6]:
import requests as r
from bs4 import BeautifulSoup as bs

In [87]:
url="https://www.airnav.com/airport/00AK"

In [95]:
response = r.get(url)
response.status_code

200

In [52]:
soup = bs(response.text, 'html.parser')
tables = soup.find_all("td", width=94)
minLL = 100
minJA = 100
for a in tables:
    rows = a.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        clean = [x for x in cols if x]
        print(clean)
        if len(clean)==3:
            if clean[0]=='FS' or clean[0]=='SS':
                LL = float(clean[1][1:])
                JA = float(clean[2][1:])
                print(LL,JA)
                if LL<minLL:
                    minLL=LL
                if JA<minJA:
                    minJA=JA
strLL = str(minLL)
strJA = str(minJA)
if len(strLL)==3:
    strLL = strLL+'0'
if len(strJA)==3:
    strJA = strJA+'0'

100 100
1000 1000


In [177]:
#100LL will be index 1, JetA index 2
#param: icao code
#returns strLL, strJA

def get_min_price(airport):
    url="https://www.airnav.com/airport/"+airport
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = r.get(url, headers=headers)
    soup = bs(response.text, 'html.parser')
    tables = soup.find_all("td", width=94)
    minLL = 100
    minJA = 100
    for a in tables:
        rows = a.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [element.text.strip() for element in cols]
            clean = [x for x in cols if x]
            if len(clean)==3:
                if clean[0]=='FS' or clean[0]=='SS':
                    if clean[1][1:]=='--':
                        LL=100
                    else:
                        LL = float(clean[1][1:])
                    if clean[2][1:]=='--':
                        JA=100
                    else:
                        JA = float(clean[2][1:])
                    if LL<minLL:
                        minLL=LL
                    if JA<minJA:
                        minJA=JA
    strLL = str(minLL)
    strJA = str(minJA)
    if len(strLL)==3:
        strLL = strLL+'0'
    if len(strJA)==3:
        strJA = strJA+'0'
        
    return strLL,strJA,response.status_code

In [107]:
get_min_price('KTEB')

('5.70', '3.89', 200)

In [110]:
import time

LLlist = []
JAlist = []
count=1
for airport in airports['icao']:
    time.sleep(5)
    results = get_min_price(airport)
    LLlist.append(results[0])
    JAlist.append(results[1])
    print('Done:',airport,results,count)
    count+=1
    
print(len(LLlist),len(JAlist))

Done: 00AK ('1000', '1000', 200) 1
Done: 00AL ('1000', '1000', 200) 2
Done: 00AZ ('1000', '1000', 200) 3
Done: 00CA ('1000', '1000', 200) 4
Done: 00CO ('1000', '1000', 200) 5
Done: 00FA ('1000', '1000', 200) 6
Done: 00FL ('1000', '1000', 200) 7
Done: 00GA ('1000', '1000', 200) 8
Done: 00ID ('1000', '1000', 200) 9
Done: 00IL ('1000', '1000', 200) 10
Done: 00IS ('1000', '1000', 200) 11
Done: 00KS ('1000', '1000', 200) 12
Done: 00KY ('1000', '1000', 200) 13
Done: 00LS ('1000', '1000', 200) 14
Done: 00MD ('1000', '1000', 200) 15
Done: 00MN ('1000', '1000', 200) 16
Done: 00MO ('1000', '1000', 200) 17
Done: 00MT ('1000', '1000', 200) 18
Done: 00NC ('1000', '1000', 200) 19
Done: 00NY ('1000', '1000', 200) 20
Done: 00OH ('1000', '1000', 200) 21
Done: 00PN ('1000', '1000', 200) 22
Done: 00PS ('1000', '1000', 200) 23
Done: 00SC ('1000', '1000', 200) 24
Done: 00TN ('1000', '1000', 200) 25
Done: 00TS ('1000', '1000', 200) 26
Done: 00VA ('1000', '1000', 200) 27
Done: 00VI ('1000', '1000', 200) 28
D

KeyboardInterrupt: 

In [139]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

def get_airports(state):
    url='https://www.airnav.com/airports/us/'+state
    response = r.get(url)
    soup = bs(response.text, 'html.parser')
    tables = soup.find_all("td")
    names = []
    for a in tables:
        rows = a.find_all('a')
        if rows:
            names.append(rows[0])
    names1 = [element.text.strip() for element in names]
    return names1[5:-2]

In [140]:
airport_dict = {}
for state in states:
    time.sleep(5)
    airport_dict[state] = get_airports(state)
    print('Done:',state)
airport_dict

Done: AL
Done: AK
Done: AZ
Done: AR
Done: CA
Done: CO
Done: CT
Done: DC
Done: DE
Done: FL
Done: GA
Done: HI
Done: ID
Done: IL
Done: IN
Done: IA
Done: KS
Done: KY
Done: LA
Done: ME
Done: MD
Done: MA
Done: MI
Done: MN
Done: MS
Done: MO
Done: MT
Done: NE
Done: NV
Done: NH
Done: NJ
Done: NM
Done: NY
Done: NC
Done: ND
Done: OH
Done: OK
Done: OR
Done: PA
Done: RI
Done: SC
Done: SD
Done: TN
Done: TX
Done: UT
Done: VT
Done: VA
Done: WA
Done: WV
Done: WI
Done: WY


{'AL': ['0J0',
  '2A8',
  'KEET',
  '8A0',
  'KALX',
  'KAIV',
  '79J',
  'KANB',
  '1M3',
  '26A',
  '0R1',
  'KAUO',
  '1R8',
  '5R7',
  'KEKY',
  'KBHM',
  '12J',
  '09A',
  '61A',
  'KPYP',
  '0A8',
  '5R1',
  '02A',
  '11A',
  '9A4',
  '15A',
  'KCMD',
  '4R9',
  'KDCU',
  'KDYA',
  'KDHN',
  '3M2',
  '14J',
  'KEDN',
  'KEUF',
  'KGZH',
  'KCQF',
  'M95',
  '0J4',
  '5R4',
  '4A9',
  'KGAD',
  '33J',
  '7A0',
  'KPRN',
  'KJKA',
  '8A1',
  '1M4',
  'KHAB',
  '5M0',
  'M38',
  '0J6',
  'KHSV',
  'KMDQ',
  '3M5',
  '4R3',
  'KJFX',
  '7A3',
  '04A',
  'A08',
  'KMOB',
  'KBFM',
  'KMVC',
  'KMGM',
  'KMSL',
  '20A',
  '71J',
  'KPLR',
  '1A9',
  '3M8',
  '7A5',
  'M22',
  '1A4',
  '4A6',
  'KSEM',
  '2R5',
  '7A6',
  'KSCD',
  'KASN',
  'KTOI',
  'KTCL',
  '06A',
  '07A',
  'M55',
  '08A'],
 'AK': ['PADK',
  'PAKH',
  'Z13',
  'PFAK',
  'PAUT',
  'PAUK',
  '5A8',
  'Z25',
  'PFAL',
  'PAFM',
  '80A',
  'PAKP',
  'PANC',
  'PAMR',
  'PALH',
  'A13',
  'PANI',
  'PANV',
  'PARC',
  '

In [148]:
USdict = {}
count=1
for key in airport_dict.keys():
    for airport in airport_dict[key]:
        time.sleep(5)
        results = get_min_price(airport)
        USdict[airport] = results
        print('Done:',airport,results,count)
        count+=1

Done: 0J0 ('1000', '1000', 200) 1
Done: 2A8 ('1000', '1000', 200) 2
Done: KEET ('3.80', '4.15', 200) 3
Done: 8A0 ('4.45', '4.10', 200) 4
Done: KALX ('3.60', '4.10', 200) 5
Done: KAIV ('1000', '1000', 200) 6
Done: 79J ('2.99', '2.29', 200) 7
Done: KANB ('3.79', '3.24', 200) 8
Done: 1M3 ('1000', '1000', 200) 9
Done: 26A ('1000', '1000', 200) 10
Done: 0R1 ('1000', '1000', 200) 11
Done: KAUO ('3.50', '3.30', 200) 12
Done: 1R8 ('1000', '3.85', 200) 13
Done: 5R7 ('1000', '1000', 200) 14
Done: KEKY ('4.48', '4.59', 200) 15
Done: KBHM ('4.83', '5.40', 200) 16
Done: 12J ('4.78', '3.90', 200) 17
Done: 09A ('1000', '1000', 200) 18
Done: 61A ('1000', '1000', 200) 19
Done: KPYP ('1000', '1000', 200) 20
Done: 0A8 ('1000', '1000', 200) 21
Done: 5R1 ('1000', '1000', 200) 22
Done: 02A ('4.45', '3.45', 200) 23
Done: 11A ('1000', '1000', 200) 24
Done: 9A4 ('1000', '1000', 200) 25
Done: 15A ('1000', '1000', 200) 26
Done: KCMD ('3.50', '2.99', 200) 27
Done: 4R9 ('1000', '1000', 200) 28
Done: KDCU ('4.50', 

KeyboardInterrupt: 

In [157]:
isDone = False
while not isDone:
    print('Trying Again')
    if len(USdict)>4759:
        isDone = True
    initkeys = list(USdict.keys())
    for key in initkeys:
        if USdict[key][2]==200:
            continue
        elif USdict[key][2]==503:
            USdict.pop(key,'None')
    isDone=True

Trying Again


In [186]:
notdone = {}
for key in airport_dict.keys():
    for ap in airport_dict[key]:
        notdone[ap] = 0


for key in USdict.keys():
    notdone.pop(key,'None')
print(len(notdone),len(USdict))

4213 540


In [187]:
count=1
iterbreak=0
while len(notdone)>0:
    if iterbreak>100:
        break
    iterbreak+=1
    for key in airport_dict.keys():
        for airport in airport_dict[key]:
            if airport in USdict:
                continue
            time.sleep(5)
            results = get_min_price(airport)
            if results[2]==503:
                print('waiting')
                time.sleep(30)
                continue
            USdict[airport] = results
            print('Done:',airport,results,count)
            count+=1
            notdone.pop(airport,'None')
        

Done: KARG ('3.44', '3.15', 200) 1
Done: L70 ('1000', '1000', 200) 2
Done: KAAT ('1000', '4.74', 200) 3
Done: A24 ('1000', '1000', 200) 4
Done: 2O3 ('1000', '1000', 200) 5
Done: KAPV ('3.45', '2.98', 200) 6
Done: KACV ('1000', '4.76', 200) 7
Done: KMER ('5.47', '3.99', 200) 8
waiting
Done: KAVX ('1000', '1000', 200) 9
Done: 0O2 ('1000', '1000', 200) 10
Done: KBFL ('5.58', '4.62', 200) 11
Done: L45 ('3.58', '2.75', 200) 12
Done: KBNG ('1000', '1000', 200) 13
Done: O02 ('1000', '1000', 200) 14
Done: O55 ('1000', '1000', 200) 15
Done: L35 ('3.67', '3.19', 200) 16
Done: KBIH ('4.37', '4.17', 200) 17
waiting
Done: D83 ('1000', '1000', 200) 18
Done: L08 ('1000', '1000', 200) 19
Done: KBWC ('3.50', '3.79', 200) 20
Done: O57 ('1000', '1000', 200) 21
Done: KBUR ('6.55', '4.28', 200) 22
Done: L62 ('1000', '1000', 200) 23
Done: C83 ('1000', '1000', 200) 24
waiting
Done: L71 ('1000', '1000', 200) 25
Done: KCLR ('1000', '1000', 200) 26
Done: KCMA ('4.70', '2.75', 200) 27
Done: O61 ('1000', '1000', 

waiting
Done: KITR ('4.35', '3.40', 200) 223
Done: 5V4 ('1000', '1000', 200) 224
Done: 1V6 ('3.95', '3.45', 200) 225
Done: 1V8 ('1000', '1000', 200) 226
Done: KCOS ('4.59', '3.73', 200) 227
Done: KFLY ('1000', '1000', 200) 228
Done: KCEZ ('5.70', '4.95', 200) 229
Done: KCAG ('5.00', '4.00', 200) 230
Done: 99V ('1000', '1000', 200) 231
waiting
Done: KRCV ('1000', '1000', 200) 232
Done: KAJZ ('4.15', '3.35', 200) 233
Done: D17 ('1000', '1000', 200) 234
Done: KAPA ('4.42', '3.74', 200) 235
Done: KBJC ('3.99', '3.74', 200) 236
Done: KDEN ('7.00', '6.31', 200) 237
Done: KCFO ('4.50', '4.44', 200) 238
Done: 8V6 ('1000', '1000', 200) 239
Done: KDRO ('5.98', '6.25', 200) 240
waiting
Done: 9V7 ('1000', '1000', 200) 241
Done: KEGE ('4.42', '3.74', 200) 242
Done: CO4 ('1000', '1000', 200) 243
Done: KEIK ('3.80', '3.23', 200) 244
Done: KFNL ('4.45', '3.66', 200) 245
Done: KFMM ('1000', '3.65', 200) 246
Done: KGWS ('4.00', '4.98', 200) 247
waiting
Done: KGJT ('4.20', '4.50', 200) 248
Done: KGXY ('3

Done: KBIJ ('3.74', '3.90', 200) 441
Done: KBQK ('4.00', '3.65', 200) 442
Done: 82A ('1000', '1000', 200) 443
Done: 6A1 ('1000', '1000', 200) 444
Done: 70J ('1000', '1000', 200) 445
Done: KCZL ('1000', '1000', 200) 446
Done: KCXU ('3.95', '3.90', 200) 447
Done: 18A ('3.99', '3.49', 200) 448
waiting
Done: KCTJ ('1000', '1000', 200) 449
Done: KVPC ('4.95', '3.25', 200) 450
Done: 4A4 ('3.65', '3.20', 200) 451
Done: KCWV ('1000', '1000', 200) 452
Done: 48A ('1000', '1000', 200) 453
Done: KCSG ('4.03', '4.65', 200) 454
Done: KCKF ('4.57', '2.65', 200) 455
waiting
Done: 25J ('1000', '1000', 200) 456
Done: 9A0 ('1000', '1000', 200) 457
Done: KDNN ('3.99', '3.49', 200) 458
Done: 16J ('1000', '1000', 200) 459
Done: 17J ('1000', '1000', 200) 460
Done: KDQH ('4.25', '3.80', 200) 461
Done: KDBN ('4.25', '4.00', 200) 462
Done: KEZM ('1000', '4.19', 200) 463
Done: KEBA ('1000', '1000', 200) 464
waiting
Done: KFZG ('4.52', '4.40', 200) 465
Done: 3J6 ('1000', '1000', 200) 466
Done: KLHW ('3.98', '4.50

Done: 06C ('5.47', '5.69', 200) 661
Done: KUGN ('4.75', '5.19', 200) 662
Done: KDPA ('4.46', '4.33', 200) 663
waiting
Done: C82 ('1000', '1000', 200) 664
Done: KDNV ('4.07', '3.90', 200) 665
Done: KDKB ('3.10', '2.70', 200) 666
Done: KDEC ('5.74', '4.79', 200) 667
Done: C73 ('1000', '1000', 200) 668
Done: KDTG ('1000', '1000', 200) 669
Done: 1H2 ('4.92', '3.83', 200) 670
Done: KFWC ('4.75', '3.75', 200) 671
Done: KFOA ('5.00', '4.00', 200) 672
waiting
Done: C86 ('1000', '1000', 200) 673
Done: KGBG ('3.99', '3.49', 200) 674
Done: 3G8 ('1000', '1000', 200) 675
Done: C81 ('1000', '1000', 200) 676
Done: KGRE ('1000', '1000', 200) 677
Done: 10C ('1000', '1000', 200) 678
Done: KHSB ('4.55', '3.80', 200) 679
Done: 0C0 ('1000', '1000', 200) 680
waiting
Done: H07 ('1000', '1000', 200) 681
Done: 0C2 ('1000', '1000', 200) 682
Done: KIJX ('1000', '1000', 200) 683
Done: KJOT ('1000', '1000', 200) 684
Done: KIKK ('4.79', '3.95', 200) 685
Done: 3KK ('1000', '1000', 200) 686
Done: KEZI ('1000', '1000'

Done: C17 ('1000', '1000', 200) 880
waiting
Done: KMCW ('3.85', '3.80', 200) 881
Done: KSXK ('1000', '1000', 200) 882
Done: 4D8 ('1000', '1000', 200) 883
Done: 7C3 ('1000', '1000', 200) 884
Done: 7C5 ('1000', '1000', 200) 885
Done: KMXO ('1000', '1000', 200) 886
Done: 1Y3 ('1000', '1000', 200) 887
Done: KMPZ ('5.35', '4.10', 200) 888
waiting
Done: 1Y5 ('1000', '1000', 200) 889
Done: KTNU ('4.95', '4.59', 200) 890
Done: 5D2 ('1000', '1000', 200) 891
Done: KOLZ ('5.05', '3.85', 200) 892
Done: D02 ('1000', '1000', 200) 893
Done: I75 ('1000', '1000', 200) 894
Done: KOOA ('3.75', '4.50', 200) 895
Done: KOTM ('3.60', '2.98', 200) 896
Done: 1Y9 ('1000', '1000', 200) 897
waiting
Done: KPRO ('4.10', '3.40', 200) 898
Done: KPOH ('1000', '1000', 200) 899
Done: 2Y0 ('1000', '1000', 200) 900
Done: 2Y1 ('1000', '1000', 200) 901
Done: KRDK ('1000', '1000', 200) 902
Done: 8Y8 ('1000', '1000', 200) 903
Done: KRRQ ('1000', '1000', 200) 904
waiting
Done: KSKI ('1000', '1000', 200) 905
Done: KSHL ('3.63',

...

Done: T31 ('1000', '1000', 200) 3235
waiting
Done: F21 ('1000', '1000', 200) 3236
Done: T50 ('1000', '1000', 200) 3237
Done: KHQZ ('3.20', '3.00', 200) 3238
Done: KLXY ('1000', '1000', 200) 3239
Done: 3E0 ('1000', '1000', 200) 3240
Done: KMAF ('3.53', '3.77', 200) 3241
Done: KMDD ('4.01', '3.94', 200) 3242
Done: 7T7 ('3.71', '2.39', 200) 3243
Done: T56 ('1000', '1000', 200) 3244
waiting
Done: 3F9 ('1000', '1000', 200) 3245
Done: KJDD ('3.24', '4.60', 200) 3246
Done: KMWL ('3.39', '3.49', 200) 3247
Done: E01 ('1000', '1000', 200) 3248
Done: F85 ('1000', '1000', 200) 3249
Done: KOSA ('1000', '2.98', 200) 3250
Done: 6X0 ('1000', '1000', 200) 3251
waiting
Done: 2T1 ('1000', '1000', 200) 3252
Done: 37F ('3.00', '4.00', 200) 3253
Done: KOCH ('3.69', '3.29', 200) 3254
Done: 60R ('5.34', '3.70', 200) 3255
Done: KBAZ ('3.99', '3.20', 200) 3256
Done: 61R ('1000', '1000', 200) 3257
Done: KODO ('3.53', '3.77', 200) 3258
Done: KONY ('1000', '1000', 200) 3259
Done: KORG ('2.58', '2.40', 200) 3260
wa

Done: W24 ('1000', '1000', 200) 3449
Done: KMKJ ('4.65', '3.75', 200) 3450
Done: KMTV ('4.19', '3.95', 200) 3451
Done: KMFV ('1000', '4.18', 200) 3452
Done: W91 ('1000', '1000', 200) 3453
waiting
Done: KPHF ('5.00', '4.50', 200) 3454
Done: KORF ('4.69', '3.49', 200) 3455
Done: KPVG ('4.69', '3.49', 200) 3456
Done: KCPK ('4.69', '3.39', 200) 3457
Done: KOMH ('1000', '1000', 200) 3458
Done: KPTB ('4.20', '3.60', 200) 3459
Done: W96 ('1000', '1000', 200) 3460
waiting
Done: KRIC ('5.67', '3.53', 200) 3461
Done: KFCI ('5.50', '4.00', 200) 3462
Done: KOFP ('4.55', '3.55', 200) 3463
Done: KROA ('5.80', '5.36', 200) 3464
Done: W75 ('1000', '1000', 200) 3465
Done: W78 ('1000', '1000', 200) 3466
Done: KAVC ('4.70', '3.89', 200) 3467
Done: KRMN ('4.50', '3.75', 200) 3468
Done: KSHD ('5.23', '4.59', 200) 3469
waiting
Done: KTGI ('1000', '1000', 200) 3470
Done: KXSA ('1000', '1000', 200) 3471
Done: KAKQ ('1000', '1000', 200) 3472
Done: KHWY ('3.75', '2.50', 200) 3473
Done: W13 ('1000', '1000', 200)

Done: KMSN ('4.39', '3.99', 200) 3663
Done: 87Y ('1000', '1000', 200) 3664
Done: D25 ('1000', '1000', 200) 3665
Done: KMTW ('1000', '1000', 200) 3666
Done: KMFI ('1000', '1000', 200) 3667
Done: KMDZ ('1000', '1000', 200) 3668
waiting
Done: KRRL ('1000', '1000', 200) 3669
Done: C29 ('3.75', '3.05', 200) 3670
Done: KMKE ('4.65', '6.19', 200) 3671
Done: KMWC ('4.99', '4.35', 200) 3672
Done: KMRJ ('1000', '1000', 200) 3673
Done: KARV ('4.74', '3.79', 200) 3674
Done: KEFT ('3.00', '2.70', 200) 3675
Done: KCWA ('4.09', '3.74', 200) 3676
Done: KDAF ('1000', '1000', 200) 3677
waiting
Done: KVIQ ('1000', '1000', 200) 3678
Done: 8D1 ('1000', '1000', 200) 3679
Done: 82C ('4.15', '3.75', 200) 3680
Done: KRNH ('1000', '1000', 200) 3681
Done: KOCQ ('1000', '1000', 200) 3682
Done: KOEO ('3.65', '3.50', 200) 3683
Done: KOSH ('4.59', '4.19', 200) 3684
Done: 88C ('1000', '1000', 200) 3685
Done: KPKF ('1000', '1000', 200) 3686
waiting
Done: KPVB ('4.05', '3.00', 200) 3687
Done: C47 ('3.74', '4.55', 200) 

Done: 27K ('3.25', '2.40', 200) 3876
Done: K24 ('1000', '1000', 200) 3877
Done: 1A6 ('4.50', '3.75', 200) 3878
Done: 18I ('1000', '1000', 200) 3879
waiting
Done: 2R6 ('1000', '1000', 200) 3880
Done: 4R7 ('1000', '1000', 200) 3881
Done: 1R1 ('1000', '1000', 200) 3882
Done: 3F3 ('3.44', '3.65', 200) 3883
Done: KMSY ('5.64', '4.45', 200) 3884
Done: KAPS ('3.36', '3.20', 200) 3885
Done: L83 ('1000', '1000', 200) 3886
Done: KAUG ('4.95', '3.95', 200) 3887
Done: B21 ('4.59', '3.49', 200) 3888
waiting
Done: 65B ('1000', '1000', 200) 3889
Done: KPQI ('1000', '2.50', 200) 3890
Done: KWVL ('5.05', '4.80', 200) 3891
Done: W32 ('1000', '4.79', 200) 3892
Done: KVKX ('1000', '1000', 200) 3893
Done: KRJD ('1000', '1000', 200) 3894
Done: KBVY ('4.45', '4.25', 200) 3895
Done: 1B6 ('1000', '1000', 200) 3896
waiting
Done: KCEF ('5.23', '4.25', 200) 3897
Done: KAMN ('1000', '1000', 200) 3898
Done: KBTL ('4.39', '2.99', 200) 3899
Done: 6Y1 ('1000', '1000', 200) 3900
Done: KFPK ('1000', '1000', 200) 3901
Do

waiting
Done: F00 ('1000', '1000', 200) 4090
Done: KBWD ('3.75', '3.76', 200) 4091
Done: KCVB ('3.50', '2.95', 200) 4092
Done: KCPT ('2.80', '3.00', 200) 4093
Done: KMKN ('3.72', '3.69', 200) 4094
Done: 8F5 ('1000', '1000', 200) 4095
Done: 76T ('1000', '1000', 200) 4096
waiting
Done: F41 ('3.05', '2.59', 200) 4097
Done: KFTW ('5.10', '4.40', 200) 4098
Done: X54 ('3.75', '3.80', 200) 4099
Done: T37 ('1000', '1000', 200) 4100
Done: 2G5 ('1000', '1000', 200) 4101
Done: KRFI ('1000', '1000', 200) 4102
Done: KSGR ('3.23', '3.60', 200) 4103
Done: KUTS ('3.51', '3.88', 200) 4104
Done: 16X ('1000', '1000', 200) 4105
waiting
Done: 3R9 ('1000', '1000', 200) 4106
Done: 00R ('1000', '1000', 200) 4107
Done: T91 ('1000', '1000', 200) 4108
Done: 2E7 ('1000', '1000', 200) 4109
Done: KJWY ('4.09', '4.01', 200) 4110
Done: F53 ('4.09', '3.80', 200) 4111
Done: KOZA ('4.35', '3.95', 200) 4112
Done: KPEQ ('3.85', '3.55', 200) 4113
Done: KPIL ('4.54', '2.80', 200) 4114
waiting
Done: T48 ('1000', '1000', 200)

In [190]:
'''
Done: KASN ('1000', '1000', 503) 79
Done: KTOI ('1000', '1000', 503) 80
Done: KTCL ('1000', '1000', 503) 81
Done: 06A ('1000', '1000', 503) 82
Done: 07A ('1000', '1000', 503) 83
Done: M55 ('1000', '1000', 503) 84
Done: 08A ('1000', '1000', 503) 85
'''
count=1
iterbreak=0
while len(notdone)>0:
    if iterbreak>100:
        break
    iterbreak+=1
    for key in range(1):
        for airport in ['KASN','KTOI','KTCL','06A','07A','M55','08A']:
            if airport in USdict:
                continue
            time.sleep(5)
            results = get_min_price(airport)
            if results[2]==503:
                print('waiting')
                time.sleep(30)
                continue
            USdict[airport] = results
            print('Done:',airport,results,count)
            count+=1
            notdone.pop(airport,'None')

In [188]:
len(USdict)

4753

In [194]:
usdf = pd.DataFrame.from_dict(USdict, orient='index')
usdf.to_csv('fueldata.csv')

In [213]:
usdf1 = usdf[(usdf[0]!='1000') | (usdf[1]!='1000')]
print(len(usdf1))

1992


In [214]:
reindex = []
for ap in usdf1.index:
    if len(ap)<3 or len(ap)>4:
        print(ap)
    if len(ap)==3:
        reindex.append('K'+ap)
    else:
        reindex.append(ap)
        
print(len(reindex))

1992


In [215]:
usdf1['icao'] = reindex
usdf1

C:\Users\Alan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,0,1,2,icao
KEET,3.80,4.15,200,KEET
8A0,4.45,4.10,200,K8A0
KALX,3.60,4.10,200,KALX
79J,2.99,2.29,200,K79J
KANB,3.79,3.24,200,KANB
...,...,...,...,...
KFVX,4.80,4.14,200,KFVX
KYKM,5.66,3.93,200,KYKM
KPNA,1000,4.05,200,KPNA
KGLR,4.80,4.94,200,KGLR


In [221]:
usdf1.columns = ['LL','JA','code','icao']
usdf1 = usdf1.drop(columns=['code'])

In [224]:
airports = airports.drop(columns=['LL','JA'])
airports

,city,icao,lat,lon,name,state
00AK,Anchor Point,00AK,59.949200,-151.695999,Lowell Field,Alaska
00AL,Harvest,00AL,34.864799,-86.770302,Epps Airpark,Alabama
00AZ,Cordes,00AZ,34.305599,-112.165001,Cordes Airport,Arizona
00CA,Barstow,00CA,35.350498,-116.888000,Goldstone /Gts/ Airport,California
00CO,Briggsdale,00CO,40.622200,-104.344002,Cass Field,Colorado
...,...,...,...,...,...,...
XS92,Paradise,XS92,33.146000,-97.712502,Jackson /Bill/ Airport,Texas
XS93,Santa Elena,XS93,26.716999,-98.558601,Diamond O Ranch Airport,Texas
XS94,San Ygnacio,XS94,27.111401,-99.422501,Corralitos Airport,Texas
XS95,Schulenburg,XS95,29.676901,-96.970497,Marty Ranch Airport,Texas


In [226]:
usdf1 = usdf1.replace(to_replace='1000', value='N/A')

In [232]:
prices = pd.merge(airports, usdf1, on='icao')

In [246]:
prices.to_csv('cleanfueldata.csv')

In [242]:
testfile = open('test.json', 'r')

In [243]:
strtest = testfile.read()

In [244]:
strtest

'eqfeed_callback({"type":"FeatureCollection", "features":[\n\t{"type":"Feature","properties":\n\t{\n        "icao": "00AK",\n        "iata": "",\n        "name": "Lowell Field",\n        "city": "Anchor Point",\n        "state": "Alaska", \n        "country": "US",\n        "elevation": 450,\n        "lat": 59.94919968,\n        "lon": -151.695999146,\n        "tz": "America\\/Anchorage",\n        "LL": "5.50",\n        "JetA" : "6.25"\n    }\n\t},\n\t{"type":"Feature","properties":\n\t{\n        "icao": "00AL",\n        "iata": "",\n        "name": "Epps Airpark",\n        "city": "Harvest",\n        "state": "Alabama",\n        "country": "US",\n        "elevation": 820,\n        "lat": 34.8647994995,\n        "lon": -86.7703018188,\n        "tz": "America\\/Chicago",\n        "LL": "5.50",\n        "JetA" : "6.25"\n    }\n\t}\n]});'

In [249]:
testdf = prices.loc[:3]

In [269]:
def build_file(df):
    flist = ["city","icao","lat","lon","name","state","LL","JetA"]
    strfile = 'eqfeed_callback({"type":"FeatureCollection", "features":['
    for row in df.iterrows():
        strap = '\n\t{"type":"Feature","properties":\n\t{\n'
        tup = row[1]
        for i in range(len(tup)):
            if flist[i]=='lat' or flist[i]=='lon':
                strap+='"'+flist[i]+'": '+str(tup[i])+',\n '
            else:
                strap+='"'+flist[i]+'": "'+tup[i]+'",\n '
        strap+='}\n\t},'
        strfile+=strap
    return strfile[:-1]+'\n]});'        

In [261]:
for row in testdf.iterrows():
    for element in row[1]:
        print(element)

Broadus
K00F
45.47249985
-105.4540024
Broadus Airport
Montana
4.50
3.77
Belmont
K01M
34.4916992188
-88.2011032104
Tishomingo County Airport
Mississippi
4.25
3.00
Clanton
K02A
32.8504981995
-86.6113967896
Gragg-Wade Field
Alabama
4.45
3.45
Eureka
K05U
39.6041984558
-116.0049972534
Eureka Airport
Nevada
5.11
4.31


In [265]:
strtest

'eqfeed_callback({"type":"FeatureCollection", "features":[\n\t{"type":"Feature","properties":\n\t{\n        "icao": "00AK",\n        "iata": "",\n        "name": "Lowell Field",\n        "city": "Anchor Point",\n        "state": "Alaska", \n        "country": "US",\n        "elevation": 450,\n        "lat": 59.94919968,\n        "lon": -151.695999146,\n        "tz": "America\\/Anchorage",\n        "LL": "5.50",\n        "JetA" : "6.25"\n    }\n\t},\n\t{"type":"Feature","properties":\n\t{\n        "icao": "00AL",\n        "iata": "",\n        "name": "Epps Airpark",\n        "city": "Harvest",\n        "state": "Alabama",\n        "country": "US",\n        "elevation": 820,\n        "lat": 34.8647994995,\n        "lon": -86.7703018188,\n        "tz": "America\\/Chicago",\n        "LL": "5.50",\n        "JetA" : "6.25"\n    }\n\t}\n]});'

In [270]:
build_file(testdf)

'eqfeed_callback({"type":"FeatureCollection", "features":[\n\t{"type":"Feature","properties":\n\t{\n"city": "Broadus",\n "icao": "K00F",\n "lat": 45.47249985,\n "lon": -105.4540024,\n "name": "Broadus Airport",\n "state": "Montana",\n "LL": "4.50",\n "JetA": "3.77",\n }\n\t},\n\t{"type":"Feature","properties":\n\t{\n"city": "Belmont",\n "icao": "K01M",\n "lat": 34.4916992188,\n "lon": -88.2011032104,\n "name": "Tishomingo County Airport",\n "state": "Mississippi",\n "LL": "4.25",\n "JetA": "3.00",\n }\n\t},\n\t{"type":"Feature","properties":\n\t{\n"city": "Clanton",\n "icao": "K02A",\n "lat": 32.8504981995,\n "lon": -86.6113967896,\n "name": "Gragg-Wade Field",\n "state": "Alabama",\n "LL": "4.45",\n "JetA": "3.45",\n }\n\t},\n\t{"type":"Feature","properties":\n\t{\n"city": "Eureka",\n "icao": "K05U",\n "lat": 39.6041984558,\n "lon": -116.0049972534,\n "name": "Eureka Airport",\n "state": "Nevada",\n "LL": "5.11",\n "JetA": "4.31",\n }\n\t}\n]});'

In [271]:
test = open("test1.json", "w")
test.write(build_file(testdf))
test.close()

In [273]:
final = open("USprices.json", "w")
final.write(build_file(prices))
final.close()

In [279]:
top = pd.read_excel('busiestairports.xlsx')

In [280]:
newcode = []
for el in top['icao']:
    newcode.append('K'+el)
top['icao']=newcode
top

,icao,ops
0,KORD,919704
1,KATL,904301
2,KDFW,720007
3,KLAX,691257
4,KDEN,640098
...,...,...
515,KHTS,13088
516,KGRI,12781
517,KIFP,10520
518,KHOB,9734


In [290]:
cutprice = pd.merge(prices, top.drop(columns=['ops']), on='icao')
cutprice

,city,icao,lat,lon,name,state,LL,JA
0,Allentown,KABE,40.652100,-75.440804,Lehigh Valley International Airport,Pennsylvania,5.17,3.68
1,Abilene,KABI,32.411301,-99.681900,Abilene Regional Airport,Texas,3.99,4.49
2,Albuquerque,KABQ,35.040199,-106.609001,Albuquerque International Sunport Airport,New-Mexico,4.49,3.99
3,Albany,KABY,31.535500,-84.194504,Southwest Georgia Regional Airport,Georgia,3.70,3.05
4,Nantucket,KACK,41.253101,-70.060204,Nantucket Memorial Airport,Massachusetts,5.28,4.29
...,...,...,...,...,...,...,...,...
482,Lancaster,KWJF,34.741100,-118.219002,General WM J Fox Airfield,California,5.29,4.49
483,Fayetteville/Springdale/,KXNA,36.281898,-94.306801,Northwest Arkansas Regional Airport,Arkansas,4.68,3.25
484,Detroit,KYIP,42.237900,-83.530403,Willow Run Airport,Michigan,5.00,4.45
485,Yakima,KYKM,46.568199,-120.543999,Yakima Air Terminal McAllister Field,Washington,5.66,3.93


In [291]:
final = open("USprices.json", "w")
final.write(build_file(cutprice))
final.close()

In [294]:
finaldict = open("USdict.txt", "w")
finaldict.write(str(USdict))
finaldict.close()